In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.optimize import GradientDescentOptimizer, AdamOptimizer, SPSAOptimizer
from pennylane.templates import BasicEntanglerLayers, StronglyEntanglingLayers

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

import matplotlib.pyplot as plt

from scipy.optimize import differential_evolution

In [2]:
def create_matrix(cut_off, type):
    # Initialize a zero matrix of the specified size
    matrix = np.zeros((cut_off, cut_off), dtype=np.complex128)
    
    # Fill the off-diagonal values with square roots of integers
    for i in range(cut_off):
        if i > 0:  # Fill left off-diagonal
            if type == 'q':
                matrix[i][i - 1] = (1/np.sqrt(2)) * np.sqrt(i)  # sqrt(i) for left off-diagonal
            else:
                matrix[i][i - 1] = (1j/np.sqrt(2)) * np.sqrt(i)

        if i < cut_off - 1:  # Fill right off-diagonal
            if type == 'q':
                matrix[i][i + 1] = (1/np.sqrt(2)) * np.sqrt(i + 1)  # sqrt(i + 1) for right off-diagonal
            else:
                matrix[i][i + 1] = (-1j/np.sqrt(2)) * np.sqrt(i + 1)

    return matrix


# Function to calculate the Hamiltonian
def calculate_Hamiltonian(cut_off, potential):
    # Generate the position (q) and momentum (p) matrices
    q = create_matrix(cut_off, 'q')  # q matrix
    p = create_matrix(cut_off, 'p')  # p matrix

    # Calculate q^2 and q^3 for potential terms
    q2 = np.matmul(q, q)
    q3 = np.matmul(q2, q)

    #fermionic identity
    I_f = np.eye(2)

    #bosonic identity
    I_b = np.eye(cut_off)

    # Superpotential derivatives
    if potential == 'QHO':
        W_prime = q  # W'(q) = q
        W_double_prime = I_b #W''(q) = 1

    elif potential == 'AHO':
        W_prime = q + q3  # W'(q) = q + q^3
        W_double_prime = I_b + 3 * q2  # W''(q) = 1 + 3q^2

    elif potential == 'DW':
        W_prime = q + q2 + I_b  # W'(q) = q + q^2 + 1
        W_double_prime = I_b + 2 * q  # W''(q) = 1 + 2q

    else:
        print("Not a valid potential")
        raise

    # Kinetic term: p^2
    p2 = np.matmul(p, p)

    # Commutator term [b^†, b] = -Z
    Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
    commutator_term = np.kron(Z, W_double_prime)

    # Construct the block-diagonal kinetic term (bosonic and fermionic parts)
    # Bosonic part is the same for both, hence we use kron with the identity matrix
    kinetic_term = np.kron(I_f, p2)

    # Potential term (W' contribution)
    potential_term = np.kron(I_f, np.matmul(W_prime, W_prime))

    # Construct the full Hamiltonian
    H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)
    H_SQM[np.abs(H_SQM) < 10e-12] = 0
    
    return H_SQM

In [4]:
# Example usage for a 4x4 matrix
cut_off = 32
potential = 'AHO'
H = calculate_Hamiltonian(cut_off, potential)
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [5]:
scale = 0.25
params_shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=num_qubits)
params = scale*np.pi * pnp.random.random(size=params_shape)
params = scale * pnp.random.random(size=params_shape)

In [6]:
# Device
#dev = qml.device('qiskit.aer', wires=num_qubits, shots=10)
dev = qml.device('lightning.qubit', wires=num_qubits, shots=1024)

# Define the cost function
@qml.qnode(dev)
def cost_function(params):
    params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
    qml.StronglyEntanglingLayers(weights=params, wires=range(num_qubits), imprimitive=qml.CZ)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [ ]:
# Define bounds for each parameter based on expected ranges (e.g., [0, 2π] for rotation angles)
bounds = [(0, 2 * np.pi) for _ in range(np.prod(params_shape))]

# Differential Evolution optimization
result = res = differential_evolution(cost_function, 
                                            bounds, 
                                            maxiter=500, 
                                            tol=1e-2,
                                            #strategy='best1bin', 
                                            strategy='rand1bin',
                                            popsize=15)
params = result.x  # Extract optimized parameters
params = pnp.tensor(params.reshape(params_shape), requires_grad=True)

KeyboardInterrupt: 

In [8]:
result

             message: Maximum number of iterations has been exceeded.
             success: False
                 fun: 0.04693221913502936
                   x: [ 4.429e+00  2.946e+00 ...  6.223e+00  5.045e+00]
                 nit: 500
                nfev: 45489
          population: [[ 4.429e+00  2.946e+00 ...  6.223e+00  5.045e+00]
                       [ 2.127e+00  3.811e+00 ...  4.587e-01  2.793e+00]
                       ...
                       [ 3.524e-01  3.820e+00 ...  6.236e+00  5.009e+00]
                       [ 2.733e+00  3.638e+00 ...  6.113e+00  4.994e+00]]
 population_energies: [ 4.693e-02  1.440e+00 ...  3.788e-01  4.411e-01]

In [17]:
stepsize = 0.5
optimizer = AdamOptimizer(stepsize=stepsize)#, beta1=0.99, beta2=0.999, eps=1e-8)

In [18]:
# Run the VQE optimization loop
max_iterations = 5000
convergence_tol = 1e-6
moving_avg_length = 5
gradient_tol = 1e-6

prev_energy = None
energies = []

moving_average_check = False
gradient_norm_check = False

for i in range(max_iterations):
    params, energy = optimizer.step_and_cost(cost_function, params)
    energies.append(energy)
    
    # Print the energy every few steps
    if i % 10 == 0:
        print(f"Iteration {i}: Energy = {energy}")

    # Moving average convergence check
    if len(energies) > moving_avg_length:
        energy_moving_avg = np.mean(np.abs(np.diff(energies[-moving_avg_length:])))
        if energy_moving_avg < convergence_tol:
            #print(f"Converged at iteration {i} with moving average change = {energy_moving_avg}")
            moving_average_check = True
            converged = True
            break

    # Gradient norm convergence check
    #grads = optimizer.compute_grad(cost_function, (params,), {})
    #grad_norm = np.linalg.norm(grads[0])
    #if grad_norm < gradient_tol:
    #    #print(f"Converged at iteration {i} with gradient norm = {grad_norm}")
    #    gradient_norm_check = True

    #if moving_average_check & gradient_norm_check:
    #    print("moving_average_check and gradient_norm_check converged")
    #    converged = True
    #    break

    # Check for convergence
    #if prev_energy is not None and np.abs(energy - prev_energy) < convergence_tol:
    #    print(f"Converged at iteration {i}")
    #    break

    prev_energy = energy

print("Final optimized energy:", energy)
print("Optimized parameters:", params)

Iteration 0: Energy = 5.312693146422113
Iteration 10: Energy = 2.0022764884396533
Iteration 20: Energy = 1.4178818944874805
Iteration 30: Energy = 1.2081819047424687
Iteration 40: Energy = 0.4029539395192246
Iteration 50: Energy = 0.2720612052565871
Iteration 60: Energy = 0.5231154169526139
Iteration 70: Energy = 0.0598739957815454
Iteration 80: Energy = 1.2530732567523881
Iteration 90: Energy = 1.6006125640843814
Iteration 100: Energy = 1.0897962324023225
Iteration 110: Energy = 2.5229179201847183
Iteration 120: Energy = 0.5050120341967193
Iteration 130: Energy = 0.7330336056617028
Iteration 140: Energy = 0.5052528193076389
Iteration 150: Energy = 0.2126987684147157
Iteration 160: Energy = 0.058930058544527104
Iteration 170: Energy = 0.14783827712735897
Iteration 180: Energy = 0.4844924746019919
Iteration 190: Energy = 0.6214260706266712
Iteration 200: Energy = 0.580063404682867
Iteration 210: Energy = 2.3493959288447326
Iteration 220: Energy = 1.45515655213253
Iteration 230: Energy =

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(energies, marker='.', linestyle='-', color='b', label="Energy")
plt.xlabel("Index")
plt.ylabel("Energy")
plt.title("Energy vs. Index")
plt.grid(True)
plt.legend()
plt.show()